In [3]:
import json
import numpy as np
import jsonlines

from collections import defaultdict

train_captions = defaultdict(list)
test_captions = defaultdict(list)

with open('/mnt/ext/phd/data/mscoco/annotations/captions_train2017.json') as f:
    train_data = json.load(f)
with open('/mnt/ext/phd/data/mscoco/annotations/captions_val2017.json') as f:
    test_data = json.load(f)
    
for row in train_data['annotations']:
    train_captions[row['image_id']].append(row['caption'].replace("\n","").strip())
for row in test_data['annotations']:
    test_captions[row['image_id']].append(row['caption'].replace("\n","").strip())
    
train_captions = list(train_captions.values())
test_captions = list(test_captions.values())
    
np.random.shuffle(train_captions)

dev_captions = train_captions[-5000:]
train_captions = train_captions[:-5000]

with jsonlines.open('../../torchseq/data/mscoco-clusters/train.jsonl', 'w') as f:
    f.write_all([{'qs': row} for row in train_captions])
with jsonlines.open('../../torchseq/data/mscoco-clusters/dev.jsonl', 'w') as f:
    f.write_all([{'qs': row} for row in dev_captions])
with jsonlines.open('../../torchseq/data/mscoco-clusters/test.jsonl', 'w') as f:
    f.write_all([{'qs': row} for row in test_captions])
    

with jsonlines.open('../../torchseq/data/mscoco-all/train.jsonl', 'w') as f:
    f.write_all([{'q': q} for row in train_captions for q in row])
with jsonlines.open('../../torchseq/data/mscoco-all/dev.jsonl', 'w') as f:
    f.write_all([{'q': q} for row in dev_captions for q in row])
with jsonlines.open('../../torchseq/data/mscoco-all/test.jsonl', 'w') as f:
    f.write_all([{'q': q} for row in test_captions for q in row])
    
    
with jsonlines.open('../../torchseq/data/mscoco-eval/train.jsonl', 'w') as f:
    f.write_all([{'sem_input': row[0], 'paras': row[1:], 'tgt': row[1], 'syn_input': row[1]} for row in train_captions])
with jsonlines.open('../../torchseq/data/mscoco-eval/dev.jsonl', 'w') as f:
    f.write_all([{'sem_input': row[0], 'paras': row[1:], 'tgt': row[1], 'syn_input': row[1]} for row in dev_captions])
with jsonlines.open('../../torchseq/data/mscoco-eval/test.jsonl', 'w') as f:
    f.write_all([{'sem_input': row[0], 'paras': row[1:], 'tgt': row[1], 'syn_input': row[1]} for row in test_captions])

In [6]:
import sacrebleu

sacrebleu.corpus_bleu

[{'sem_input': 'A black Honda motorcycle parked in front of a garage.',
  'paras': ['A Honda motorcycle parked in a grass driveway',
   'A black Honda motorcycle with a dark burgundy seat.',
   'Ma motorcycle parked on the gravel in front of a garage',
   'A motorcycle with its brake extended standing outside'],
  'tgt': 'A Honda motorcycle parked in a grass driveway',
  'syn_input': 'A Honda motorcycle parked in a grass driveway'},
 {'sem_input': 'An office cubicle with four different types of computers.',
  'paras': ['The home office space seems to be very cluttered.',
   'an office with desk computer and chair and laptop.',
   'Office setting with a lot of computer screens.',
   'A desk and chair in an office cubicle.'],
  'tgt': 'The home office space seems to be very cluttered.',
  'syn_input': 'The home office space seems to be very cluttered.'},
 {'sem_input': 'A small closed toilet in a cramped space.',
  'paras': ['A tan toilet and sink combination in a small room.',
   'This 